<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px" />

# Excel Madness Lab!

_Author:_ Tim Book

### I HAVE NOT YET FINISHED THIS LAB, I AM PLANNING ON COMING BACK AT A LATER DATE TO DO IT

## Our Mission
We work for a large supermarket chain, with stores in 10 major cities that happen to coincide with General Assembly campuses. However, this company's idea of a "database" is just a bunch of Excel spreadsheets! In order to analyze our data, we're going to need to process the existing data into a form we can use. **Our end goal is to have one csv per city.**

## Cleanup Duty!
It is a hard truth that data scientists spend a large majority of their time cleaning data. Data never arrives on our desks in exactly the format in which we want it, and it's up to us to transform it to a workable format.

Being good cleaning, moving, and reshaping data is in itself a valuable and employable job skill. If you follow these directions exactly, we will walk through constructing an automated process for processing data from this supermarket chain.

# Part I: Processing

### Step 1: Imports and the `os` library
We're going to import three libraries: numpy, pandas, and `os`.

In [52]:
clear()

In [53]:
# Import libraries here.
import numpy as np
import pandas as pd
import os as os
import matplotlib.pyplot as plt

The `os` library is extremely useful for performing system commands from within Python. Let's get two pieces of overhead out of the way now:

1. Create an `output` folder using `os.mkdir()`
2. Create a variable called `files` that is the list of files in the `data` folder using `os.listdir()`

**WARNING:** The `os.mkdir()` function will give you an error if you try to make a folder that already exists!

In [54]:
# Create an output folder.
#os.mkdir('output')

In [55]:
# Create a files variable that contains all of our data files.
files = os.listdir('data')

### Step 2: Process one data frame
It looks like we have data for the month of January. 31 files of 10 sheets each! Luckily they are all in the same format. So let's read just one in and process that. It might be helpful to open one up in your spreadsheet viewer of choice first (Excel, Numbers, Sheets, etc.)

In [56]:
# Read in data from your city from January 1st.
e_data = pd.read_excel('data/Jan 1.xlsx', 'New York')

In [57]:
e_data.head()

,prodcode,price_eu,weight_kg,quantity
0,4159,2.048141,8.812961,138
1,4062,2.728485,9.331372,184
2,4404,2.182498,8.937798,491
3,4412,2.342694,7.067682,218
4,4131,1.802839,9.551503,391


### Step 2a: Convert to 'Merican columns
For whatever reason, our data are stored in euros and kilograms. Create `price_usd` and `weight_lb` columns. There are 2.2 pounds per kilogram, and 1.1 dollars per euro.

In [58]:
# Convert our prices
e_data['price_usd'] = 1.1*e_data['price_eu']

In [59]:
# Convert our weights
e_data['weight_lb'] = 2.2*e_data['weight_kg']
e_data.head()

,prodcode,price_eu,weight_kg,quantity,price_usd,weight_lb
0,4159,2.048141,8.812961,138,2.252955,19.388515
1,4062,2.728485,9.331372,184,3.001334,20.529018
2,4404,2.182498,8.937798,491,2.400748,19.663156
3,4412,2.342694,7.067682,218,2.576964,15.548899
4,4131,1.802839,9.551503,391,1.983123,21.013306


### Step 2b: Merge in product names
You'll notice we also have a `plu-codes.csv` file containing actual product names matched up against their price lookup (PLU) codes. Let's merge these product names onto our Jan 1 data.
* _Hint 1:_ What kind of merge is this? Right, left, inner, outer, etc.?
* _Hint 2:_ Pay special attention to column names!

In [60]:
# Read in our PLU codes
plu = pd.read_csv("plu-codes.csv")

In [61]:
plu.head()

,product,plu_code
0,Apple (Fuji),4131
1,Apple (Gala),4134
2,Apricot,3302
3,Avocado,4225
4,Banana,4011


In [62]:
# Merge dataframes
df = pd.merge(e_data, plu, left_on='prodcode', right_on='plu_code')

In [63]:
df.head()

,prodcode,price_eu,weight_kg,quantity,price_usd,weight_lb,product,plu_code
0,4159,2.048141,8.812961,138,2.252955,19.388515,Onion,4159
1,4159,2.124963,1.906494,319,2.337459,4.194287,Onion,4159
2,4159,1.834220,7.788227,229,2.017642,17.134099,Onion,4159
3,4159,1.281120,4.506284,164,1.409232,9.913825,Onion,4159
4,4159,1.162582,8.225310,447,1.278840,18.095683,Onion,4159


### Step 2c: Drop unnecessary columns
We've created some extraneous columns. Drop the old price and weight columns, as well as any redundant columns.

In [64]:
#Drop unnecessary columns
df.drop('price_eu', axis=1, inplace=True)
df.drop('weight_kg', axis=1, inplace=True)
df.drop('plu_code', axis=1, inplace=True)
df.head()

,prodcode,quantity,price_usd,weight_lb,product
0,4159,138,2.252955,19.388515,Onion
1,4159,319,2.337459,4.194287,Onion
2,4159,229,2.017642,17.134099,Onion
3,4159,164,1.409232,9.913825,Onion
4,4159,447,1.278840,18.095683,Onion


### Step 2d: Add the date
Simply create a new `date` column that is the date this data was collected. For example, if this is from `Jan 1.xlsx`, this column should be full of `Jan 1`.

In [65]:
# The date for this data set is all Jan 1, we'll change this function below to generalize it
y = []
x = 'Jan 1'
for i in range(1, df.shape[0]+1):
    #print(x)
    y.append(x) 

df['date'] = y

In [66]:
df.head()

,prodcode,quantity,price_usd,weight_lb,product,date
0,4159,138,2.252955,19.388515,Onion,Jan 1
1,4159,319,2.337459,4.194287,Onion,Jan 1
2,4159,229,2.017642,17.134099,Onion,Jan 1
3,4159,164,1.409232,9.913825,Onion,Jan 1
4,4159,447,1.278840,18.095683,Onion,Jan 1


### Step 3: Write a function that conducts all of Step 2
This function should import a **filename and a city name** and return a fully processed DataFrame. That is, the function should:
1. Read in the data from the given file and city.
1. Create USD and pound columns.
1. Merge in product names.
1. Drop unnecessary columns.
1. Add a date column

In [67]:
# Combine what we did above into once function
def process_data(file, city):
    # Read in data from your city from January 1st.
    call_file = 'data/' + file
    c_data = pd.read_excel(call_file, city)
    c_data['price_usd'] = 1.1*c_data['price_eu']
    c_data['weight_lb'] = 2.2*c_data['weight_kg']
    plu = pd.read_csv("plu-codes.csv")
    df2 = pd.merge(c_data, plu, left_on='prodcode', right_on='plu_code')
    df2.drop('price_eu', axis=1, inplace=True)
    df2.drop('weight_kg', axis=1, inplace=True)
    df2.drop('plu_code', axis=1, inplace=True)
    
    y = []
    x = file.split('.')[0]
    for i in range(1, df2.shape[0]+1):
        y.append(x)
    df2['date'] = y
    return df2

Test your function out on a new file and city!

In [68]:
# Try out our function
city = 'Atlanta'
file = 'Jan 1.xlsx'
df3 = process_data(file, city)
df3.head()

,prodcode,quantity,price_usd,weight_lb,product,date
0,4272,155,1.285202,10.184810,Grapes,Jan 1
1,4272,203,1.450494,14.053250,Grapes,Jan 1
2,4272,298,0.815706,2.644650,Grapes,Jan 1
3,4272,236,1.634896,15.669850,Grapes,Jan 1
4,4272,178,1.642055,2.804827,Grapes,Jan 1


### Step 4: Process all of January's data
For each spreadsheet, process the data and store the resulting DataFrame in one big list. **You only need to do this for your city!**

* _Hint 1:_ A listcomp would make this whole step one line of code!
* _Hint 2:_ You've already made that `files` variable to help you here.

In [69]:
# Process the January data for New York
ny_data = [process_data(x, 'New York') for x in files]

### Step 5: Concatenate all DataFrames from Step 4 into one large DataFrame
* _Hint:_ Is there a function in `pandas` that can do something like this for us? This is also just one line of code!

In [70]:
# Throw it all into the same dataframe
df_ny = pd.concat(ny_data)

In [71]:
df_ny.head()

,prodcode,quantity,price_usd,weight_lb,product,date
0,4650,109,1.330281,17.027600,Mushroom,Jan 23
1,4650,107,1.979061,3.396520,Mushroom,Jan 23
2,4650,498,2.739112,5.918647,Mushroom,Jan 23
3,4650,350,1.092037,19.737597,Mushroom,Jan 23
4,4650,216,0.778506,21.484154,Mushroom,Jan 23


### Step 6: Do this for all cities, write data
Here's the big one. For each city, process and the data as in steps 3-5, and then write the data to our `output` folder. Below is a dictionary of city name to desired output file name.

Before writing your DataFrame, do the following:
* Add a `city` column
* Reorder the columns into the following order:


| city | date | product | prodcode | quantity | weight_lb | price_usd |
|---|---|---|---|---|---|---|

* _Hint:_ You can reorder DataFrame columns simply by writing over your DataFrame with itself, but specifying the specific column order with `.loc`. For example:

`print(df)`

| b | c | a |
|---|---|---|
| 1 | 2 | 3 |

`df = df.loc[:, ["a", "b", "c"]]`

`print(df)`

| a | b | c |
|---|---|---|
| 3 | 1 | 2 |


In [72]:
city_dict = {
    "Atlanta": "atl.csv",
    "Austin": "atx.csv",
    "Boston": "bos.csv",
    "Chicago": "chi.csv",
    "Denver": "den.csv",
    "Los Angeles": "lax.csv",
    "New York": "nyc.csv",
    "San Francisco": "sf.csv",
    "Seattle": "sea.csv",
    "Washington, DC": "dc.csv"
}

In [73]:
# Loop through city_dict and carry out Step 6 here.
# The keys of city_dict can serve as the sheet name.
# The values of city_dict are what you should name the output .csv files.
# If done correctly, this cell could take almost a minute to run!
def p_data(file, city):
    # Read in data from your city from January 1st.
    call_file = 'data/' + file
    c_data = pd.read_excel(call_file, city)
    c_data['price_usd'] = 1.1*c_data['price_eu']
    c_data['weight_lb'] = 2.2*c_data['weight_kg']
    plu = pd.read_csv("plu-codes.csv")
    df2 = pd.merge(c_data, plu, left_on='prodcode', right_on='plu_code')
    df2.drop('price_eu', axis=1, inplace=True)
    df2.drop('weight_kg', axis=1, inplace=True)
    df2.drop('plu_code', axis=1, inplace=True)
    
    # Here we'll redefine our date creater so it's generalized and add a column for the city
    dates = []
    cities = []
    x = file.split('.')[0]
    for i in range(1, df2.shape[0]+1):
        cities.append(city)
        dates.append(x)
    df2['date'] = dates
    df2['city'] = cities
    return df2

# Loop it for all the cities
for key, value in city_dict.items():
    city_data = [p_data(x, key) for x in files]
    df2 = pd.concat(city_data)
    df2.to_csv(f'./output/{value}', index=False)


# Part II: Checking our answers 
In steps very similar to the ones conducted above...
1. Loop through the files we just wrote to `output`, and read them in, collecting them all in one list
1. Combine all of those DataFrames into one large DataFrame
1. For each city, find the mean `quantity`, `weight_lb`, and `price_usd`.

If you've done everything correct, your answer should look exactly like this:

![](imgs/correct-output.png)

In [74]:
# Read all the data into a DataFrame
list = pd.concat([pd.read_csv(f'./output/{value}') for key, value in city_dict.items()])
    

In [75]:
list.head()

,prodcode,quantity,price_usd,weight_lb,product,date,city
0,4011,346,1.943930,7.888168,Banana,Jan 23,Atlanta
1,4011,386,2.780611,19.532345,Banana,Jan 23,Atlanta
2,4011,224,0.827797,19.813340,Banana,Jan 23,Atlanta
3,4011,394,2.942939,15.360486,Banana,Jan 23,Atlanta
4,4011,200,1.051768,14.426368,Banana,Jan 23,Atlanta


In [76]:
# Use a groupby and .mean()
list.groupby('city')['quantity', 'weight_lb', 'price_usd'].mean()

/Library/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,quantity,weight_lb,price_usd
city,,,
Atlanta,302.485306,11.929027,1.937980
Austin,301.048298,12.092775,1.937456
Boston,298.806536,12.063057,1.900973
Chicago,301.686186,12.101555,1.930026
Denver,301.012745,12.129850,1.932088
Los Angeles,301.531276,12.167527,1.918331
New York,299.482863,12.090294,1.912662
San Francisco,298.979895,12.237399,1.925830
Seattle,300.333694,11.925486,1.903315


# Part III (BONUS): Get this process production-ready!
_This part of the lab is optional, but very highly recommended, as the skills developed in this part are extremely common in industry._

For this step, we're going to take this whole process and put it into a production-ready Python script. **ABSOLUTELY NONE OF THIS STEP SHOULD TAKE PLACE IN A JUPYTER NOTEBOOK! PRODUCTIONALIZED ETL (_"Extract, Transform, Load"_) CODE DOES NOT TAKE PLACE IN A JUPYTER NOTEBOOK!**

The instructions are simple: As conducted in this lab, read, transform, and export the data in our Excel files into .csv files. The code should be in a `.py` file and executable from the command line. Here are some hints and tips to guide you:

### Hints, tips, and tricks:
* A good place to start is with the code you've already written. In this notebook, you can click `File > Download as > Python (.py)` to export as a `.py` file. Most of this exercise then comes down to you cleaning this file. (There will be a lot to clean).
* Remember `os.mkdir()` will throw an error if the folder you're trying to make already exists. Maybe you should check to see if it already exists. If it already exists, what should you do? (Remember that `.csv` can be overwritten with no problem.) The functions that can help you with this are all in the `os` library.
* Remember to follow all of the Python best practices we've already learned:
    - All import statements should go at the top of your script.
    - Comment your code. Comments shouldn't explain _what_ code does, but _why_ the code does this.
    - Keep your code DRY (don't repeat yourself) as opposed to WET (write everything twice). All constants should be variables that only need to be changed once. All code should be bottled into functions so you only need to fix it once.
* Make sure not to hardcode "Jan" anywhere. The point is that this code can be run throughout the lifetime of this supermarket's project, which is likely months or years. Keep your code so that if you get February data, you only need to change one tiny piece of the script (probably a file path).